In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import time
import sys
import tensorflow_probability as tfp

In [3]:
def build_model(width, height, depth, classes):
	# initialize the input shape
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = Sequential([
		Conv2D(30, (3, 3), padding="same", input_shape=inputShape),
		Activation("relu"),
		MaxPooling2D(pool_size=(2, 2)),
		Conv2D(50, (3, 3), padding="same"),
		Activation("relu"),
		Flatten(),
		Dense(200),
		Activation("relu"),
		# softmax classifier
		Dense(classes),
		Activation("softmax")
	])
	# return the built model to the calling function
	return model

In [4]:
def step(X, y, model):
  # keep track of our gradients
  with tf.GradientTape() as tape:
		# make a prediction using the model and then calculate the
		# loss
    pred = model(X)
    loss = categorical_crossentropy(y, pred)
	# calculate the gradients using our tape and then update the
	# model weights
  grads = tape.gradient(loss, model.trainable_variables)
  return(grads)

In [5]:
# initialize the number of epochs to train for, batch size, and
# initial learning rate
EPOCHS = 1
BS = 32
INIT_LR = 0.005
# load the MNIST dataset
print("[INFO] loading MNIST dataset...")
((trainX, trainY), (testX, testY)) = mnist.load_data()
# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
# one-hot encode the labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

[INFO] loading MNIST dataset...
11501568/11490434 [==============================] - 0s 0us/step


In [6]:
num_machines = 10
num_train = int(trainX.shape[0])
num_train_per_machine = int(num_train/num_machines)

#Split training and test data among 10 machines
for i in range(num_machines):
  globals()['X_train_%s' % str(i+1)] = trainX[i*num_train_per_machine: (i+1)*num_train_per_machine]
  globals()['y_train_%s' % str(i+1)] = trainY[i*num_train_per_machine: (i+1)*num_train_per_machine]


In [7]:
# build our model and initialize our optimizer
print("[INFO] creating model...")
model = build_model(28, 28, 1, 10)
opt = SGD(learning_rate=INIT_LR)

[INFO] creating model...


In [8]:
model.compile(optimizer=opt, loss=categorical_crossentropy,
	metrics=["acc"])
# now that the model is compiled we can compute the accuracy
(loss, acc) = model.evaluate(testX, testY)
print("[INFO] test accuracy: {:.4f}".format(acc))


313/313 [==============================] - 6s 15ms/step - loss: 2.3017 - acc: 0.0614
[INFO] test accuracy: 0.0614


In [9]:
# Uncomment for label switching attacks. The variable is a list of machines which cause the attack
# label_switch_attack_machines = [1,5]

loss_list = []
acc_list = []
for i in range(1000):
  print("------------------------"+str(i)+"-----------------------------")
  if(i%10 == 0):
    (loss, acc) = model.evaluate(testX, testY)
    print("[INFO] test accuracy: {:.4f}".format(acc))
    loss_list.append((i,loss))
    acc_list.append((i,acc))

  gradient_mean = []
  sum = [0] * 8
  flag = [0] * 8
  #For each machine, select a batch of 32 training data and perform one feedford and get gradients
  for machine in range(10):
    index = np.random.choice(globals()['X_train_%s' % str(machine+1)].shape[0], 32, replace=False)
    train_X_batch = globals()['X_train_%s' % str(machine+1)][index].copy()
    train_y_batch = globals()['y_train_%s' % str(machine+1)][index].copy()

    # Uncomment for label switching attacks 
    # if(machine in label_switch_attack_machines):
    #   #print(machine)
    #   for batch_element in range(32):
    #     for pos in range(10):
    #       if(train_y_batch_mean[batch_element][pos]==1):
    #         train_y_batch_mean[batch_element][pos]=0.
    #         train_y_batch_mean[batch_element][9-pos]=1.

    # Calculate gradients for one batch of data
    globals()['grads_mean_%s' % str(machine+1)] = step(train_X_batch, train_y_batch, model)

  #Getting gradient mean
  for machine in range(10):
    for tensor in range(len(globals()['grads_mean_%s' % str(machine+1)])):
      if(flag[tensor] == 0):
        #print(machine)
        sum[tensor] = globals()['grads_mean_%s' % str(machine+1)][tensor]
        flag[tensor] = 1
      else:
        #print(machine)
        #print(sum[tensor])
        sum[tensor] += globals()['grads_mean_%s' % str(machine+1)][tensor]
        #print(sum[tensor])

  for element in sum:
      gradient_mean.append(element/10)

  # Apply the mean gradients and carry out a single step of Gradient descent
  opt.apply_gradients(zip(gradient_mean, model.trainable_variables))


------------------------0-----------------------------
313/313 [==============================] - 5s 15ms/step - loss: 2.3017 - acc: 0.0614
[INFO] test accuracy: 0.0614
------------------------1-----------------------------
------------------------2-----------------------------
------------------------3-----------------------------
------------------------4-----------------------------
------------------------5-----------------------------
------------------------6-----------------------------
------------------------7-----------------------------
------------------------8-----------------------------
------------------------9-----------------------------
------------------------10-----------------------------
313/313 [==============================] - 5s 15ms/step - loss: 1.5812 - acc: 0.6727
[INFO] test accuracy: 0.6727
------------------------11-----------------------------
------------------------12-----------------------------
------------------------13----------------------------

In [10]:
# Test Loss and accuracy 
(loss, acc) = model.evaluate(testX, testY)
print("[INFO] test accuracy: {:.4f}".format(acc))
loss_list.append((i,loss))
acc_list.append((i,acc))

f = open("loss_mean.txt", "w")
for element in loss_list:
    f.write(str(element) + "\n")
f.close()

f = open("acc_mean.txt", "w")
for element in acc_list:
    f.write(str(element) + "\n")
f.close()

313/313 [==============================] - 5s 15ms/step - loss: 0.0496 - acc: 0.9843
[INFO] test accuracy: 0.9843


In [11]:
!pwd

/content
